In [1]:
import numpy as np
import cv2
import glob, os

In [2]:
# Basic corner detection
def detect_corners(img, number_of_corners: int, min_distance_beteen_corners: int):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    corners = cv2.goodFeaturesToTrack(gray, number_of_corners, 0.01, min_distance_beteen_corners)
    corners = np.int0([corner[0] for corner in corners])
    return corners


# Corner detection on saturation and value thresholds
def saNva_corners(img, number_of_corners: int, min_distance_beteen_corners: int):
    # Splitting HSV channels
    h, s, v = cv2.split(cv2.cvtColor(img, cv2.COLOR_BGR2HSV))

    # Blurring and thresholding saturation and value channels
    blur_s = cv2.GaussianBlur(s, (5,5), 0)
    _, th_s = cv2.threshold(blur_s, 50, 255, cv2.THRESH_BINARY_INV)
    blur_v = cv2.GaussianBlur(v, (31,31), 0)
    _, th_v = cv2.threshold(blur_v, 200, 255, cv2.THRESH_BINARY)

    # Combined threshold
    combined = cv2.addWeighted(th_s, .5, th_v, .5, 0)
    _, th = cv2.threshold(combined, 200, 255, cv2.THRESH_BINARY)

    # Corner detection
    corners = cv2.goodFeaturesToTrack(th, number_of_corners, 0.01, min_distance_beteen_corners, blockSize=25)
    if corners is not None:
        return np.int0([corner[0] for corner in corners])
    return np.int0([])

# Sort the found corners so that the first one is close to the top left, second to bottom left, third to bottom right, last to top right
def sort_corners(corner_list, shape):
    # reformatting corners as list of tuples
    corners = []
    for corn in corner_list:
        corners.append((corn[0], corn[1]))
    sorted = []
    for imgc in [(0, 0), (shape[0], 0), (shape[0], shape[1]), (0, shape[1])]:
        dists = {corn: (imgc[1] - corn[0])**2 + (imgc[0] - corn[1])**2 
                 for corn in corners}
        sorted.append(min(dists, key=dists.get))
        corners.remove(sorted[-1])
    return sorted

# Warp picture from the 4 selected corners
def warp_pic(img, corners):
    shape = img.shape
    frame = (int(shape[0]*.05)+5, int(shape[1]*.05)+5)
    output_corners = [(frame[1], frame[0]), (frame[1], shape[0]-frame[0]), (shape[1]-frame[1], shape[0]-frame[0]), (shape[1]-frame[1], frame[0])]
    M = cv2.getPerspectiveTransform(np.float32(corners), np.float32(output_corners))
    return cv2.warpPerspective(img,M,(shape[1], shape[0]),flags=cv2.INTER_LINEAR)

# Corner drawing for testing purposes
def draw_corners(corners, selected, img):
    img = img.copy()
    for corner in corners:
        x, y = corner.ravel()
        if (x, y) in selected:
            cv2.circle(img, (x, y), 10, (0, 128, 255), -1)    
        else:
            cv2.circle(img, (x, y), 5, (255, 0, 0), -1)
    
    cv2.imshow('Frame', img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


In [3]:
# Testing on one picture

test_img_path = "./data_1/color_cropped/IMG_6349_2.png"
img1 = cv2.imread(test_img_path)
corners_found = saNva_corners(img1, 100, 70)
print(corners_found)
warp_points = sort_corners(corners_found, img1.shape)
print(warp_points)
draw_corners(corners_found, warp_points, img1)

warped_pic = warp_pic(img1, warp_points)

cv2.imwrite("./test_warp.png", warped_pic)


[[ 79 527]
 [712 379]
 [ 82 201]
 [151 437]
 [497 208]
 [ 55 448]
 [460 399]
 [366 180]
 [435 148]
 [446  64]
 [666 117]
 [223  96]
 [593  91]
 [107 359]
 [318  82]
 [611 239]
 [286 456]
 [290 244]
 [172 156]
 [575 174]
 [691 449]
 [423 222]
 [ 57 595]
 [741 208]
 [871 139]
 [449 314]
 [ 28 696]
 [ 42 262]
 [849 294]
 [800 136]
 [572 497]
 [325 324]
 [647 509]
 [789 521]
 [254 161]
 [169 590]
 [649  49]
 [583  21]
 [520  71]
 [112  98]
 [123 644]
 [530 304]
 [499 600]
 [399 614]
 [213 523]
 [572 593]
 [791 344]
 [228 634]
 [663 189]
 [379  41]
 [796 445]
 [638 618]
 [367 264]
 [725 575]
 [183 688]
 [289 676]
 [ 14 391]
 [494   6]
 [321 613]
 [884 232]]
[(112, 98), (28, 696), (789, 521), (871, 139)]


True

In [65]:
# Basic crop filter, simply avoiding very small crops, could be replaced with an algorithm that quickly checks if a crop has any text
def pic_filter(img):
    if (img.shape[0] > 150 or img.shape[1] > 150) and img.shape[0]*img.shape[1] > 10000:
        return True
    return False

In [75]:
cropped1_dir = './data_1/color_cropped/'
cropped2_dir = './data_2/color_cropped/'


for data_dir in [cropped1_dir, cropped2_dir]:
    warped_dir = data_dir + 'warped'
    print(warped_dir)
    if not os.path.exists(warped_dir):
        os.makedirs(warped_dir)

    # Warping all pictures in the given directories
    for dir_path in glob.glob(data_dir):
        img_count = 0
        for img_path in glob.glob(os.path.join(dir_path, '*.png')):
            img_name = img_path.split('\\')[1][:-4]
            img = cv2.imread(img_path)
            if not pic_filter(img):
                continue
            corners_found = saNva_corners(img, 100, 70)
            if len(corners_found) < 4:
                # If the openCV algorithm did not find any corners, color and lightning conditions are probably out of range for the thresholds
                continue
            trans_points = sort_corners(corners_found, img.shape)
            warped_pic = warp_pic(img, trans_points)
            cv2.imwrite(warped_dir + '/' + img_name + f'.png', warped_pic)
            img_count+=1
            print(f'{img_count}', end=" ", flush=True)
        print(data_dir, "done.")

./data_1/color_cropped/warped
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 ./data_1/color_cropped/ done.
./data_2/color_cropped/warped
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 3